**Use Case**
A nurse practitioner wants to calculate a patient's body mass index (BMI). 


**Define Functional requirements**  
Step 1 - Define required clinical data elements  
- Patient height (inches)
- Patient weight (pounds)  
    
  
Step 2 - Identify FHIR Resources to support use case data (http://hl7.org/fhir/resourcelist.html)
- Patient height: Observation resource http://hl7.org/fhir/observation.html
- Patient weight: Observation resource http://hl7.org/fhir/observation.html

Step 3 - Identify data elements within each FHIR Resource definition necessary to support use case data


## Initialize the environment to run the application

In [1]:
# Install the Python FHIR client package in the current Jupyter kernel using pip



In [2]:
# Import the client library that understands how to make FHIR calls and interpret the results
#  Remember to import: client, datetime, requests, and json
#  Then print 'FHIR client library has been loaded'



FHIR client library has been loaded


## Load the data needed for the Framinghan analysis

In [3]:
#Load the Patient from the database, and display demographics
from dateutil.relativedelta import relativedelta  # a utility function for comparing dates
import datetime
# Define the FHIR Endpoint. We define the name of this app (used in SMART calls), and the location of the server
settings = {
    'app_id': 'my_web_app',
    'api_base': 'https://fhirtest.uhn.ca/baseDstu3'
}

In [4]:
# Create an 'instance' of the FHIR library that points to the FHIR server. We use this to make the calls.
db = client.FHIRClient(settings=settings)

In [5]:
# Perform a GET (read) on the patient with an id of 'cf-1495289345197' (a test patient we created)
# We could also search on name or any other property to find the patient
# Under the hood, a GET request is issued to the server, eg GET [server]/Patient/2789048
import fhirclient.models.patient as p
print('Reading patient, please wait...')
patient = p.Patient.read('cf-1533147316668', db.server)

Reading patient, please wait...


In [6]:
# Print the patient name. A Patient can have more than one name (ie Patient.name is multiple)
for name in patient.name:
    first_name = name.given[0]
    last_name = name.family
    print ('Patient\'s Name(s) = ',first_name,last_name)

Patient's Name(s) =  Joseph Framingham


In [7]:
# Print patient's gender (only 1 gender !)
gender = patient.gender
print ('Gender = ',gender)

Gender =  male


In [8]:
# Define variable DOB as the patient's date of birth in a string format (so it can be displayed)
DOB = patient.birthDate.isostring
print ('DOB = ', DOB)

DOB =  1961-07-31


In [9]:
# Define and print today variable. We'll use this for the age (coming up next)
now = datetime.datetime.today()
print ('Today\'s Date = ', now)

Today's Date =  2018-09-19 14:18:00.301997


In [10]:
#now get the patients age using the relativedelta library function. Don't sweat the details!
#Define DOB2 date object
DOB2 = datetime.datetime.strptime(DOB, '%Y-%m-%d')

In [11]:
# Calcuate patient's age using rdelta method of the dateutil module
rdelta = relativedelta(now, DOB2)
age=rdelta.years

# print the years from the rdelta object
print ('Patient\'s age = ',age)

Patient's age =  57


In [12]:
#All of the above scripts could have been combined into a single cell
#Below are all the print statements:
print ('Patient\'s Name(s) = ',first_name,last_name)
print ('Gender = ',gender)
print ('DOB = ', DOB)
print ('Today\'s Date = ', now)
print ('Patient\'s age = ',age)

Patient's Name(s) =  Joseph Framingham
Gender =  male
DOB =  1961-07-31
Today's Date =  2018-09-19 14:18:00.301997
Patient's age =  57


In [13]:
import fhirclient.models.observation as obs   #allows search of Observation resources
# specify the search, setting the patient, the code we want and in descending order by date
search = obs.Observation.where(struct={'patient':"Patient/2789048",'code':"3141-9",'_sort':'-date'})

#show the actual url that is generated
url = search.construct()
print('Query: ', url)

weight = search.perform_resources(db.server)
if weight:
    weight_val = weight[0].valueQuantity.value
    print(str(weight_val) + " " + weight[0].valueQuantity.unit)

Query:  Observation?patient=Patient%2Fcf-1533147316668&code=18262-6&_sort=-date
The most recent value is  189 mg/dL


In [14]:
import fhirclient.models.observation as obs   #allows search of Observation resources
# specify the search, setting the patient, the code we want and in descending order by date
search = obs.Observation.where(struct={'patient':"Patient/2789048",'code':"8302-2",'_sort':'-date'})

#show the actual url that is generated
url = search.construct()
print('Query: ', url)

height = search.perform_resources(db.server)
if height:
    height_val = height[0].valueQuantity.value
    print(str(height_val) + " " + height[0].valueQuantity.unit)

Query:  Observation?patient=Patient%2Fcf-1533147316668&code=2085-9&_sort=-date
The most recent value is  32 mg/dL


In [ ]:
bmi = weight_val/((height_val/100)**2)
bmi

Query:  Observation?patient=Patient%2Fcf-1533147316668&code=55284-4&_sort=-date
Systolic: 154 mmHG
Diastolic: 89 mmHG


In [19]:
# Does the patient have diabetes? Check if there is a Condition with a code of either Type 1 or Type 2 Diabates
# We assume a SNOMED code, note that the system for SNOMED is specified - eg http://snomed.info/sct|44054006
# In practice, you'd need to check for multiple codes - there are terminology services that make this easier
from fhirclient import client
import fhirclient.models.condition as cond
#This search is for type 1 diabetes
search1 = cond.Condition.where(struct={'patient':'Patient/2789048','code':'http://snomed.info/sct|46635009'})
print('Query: ', search1.construct())
#This is for type 2 diabetes
search2 = cond.Condition.where(struct={'patient':'Patient/2789048','code':'http://snomed.info/sct|44054006'})
print('Query: ', search2.construct())

#perform the query
type1 = search1.perform_resources(db.server)
type2 = search2.perform_resources(db.server)

if type1:
    print('Patient has type 1 diabetes')
    
if type2:
    print('Patient has type 2 diabetes')

#condition that says whether they have one of these two types of diabetes
if type1 or type2:
    diabetes = True   #this is the value that will be used in the Framingham calculation
    print("Patient has diabetes")

Query:  Condition?patient=Patient%2Fcf-1533147316668&code=http%3A%2F%2Fsnomed.info%2Fsct%7C46635009
Query:  Condition?patient=Patient%2Fcf-1533147316668&code=http%3A%2F%2Fsnomed.info%2Fsct%7C44054006
Patient has type 2 diabetes
Patient has diabetes


In [20]:
#Look for an Observation with a LOINC code for smoker
from fhirclient import client
import fhirclient.models.observation as obs
search = obs.Observation.where(struct={'patient':'Patient/2789048','code':'http://loinc.org|72166-2'})
smoking = search.perform_resources(db.server)
if smoking:
    smoke = True
    print("Patient smokes")
else:
    smoke = False
    print("Patient doesn't smoke")


Patient smokes


## Now that we have the data we need, we can perform the Framingham calculations

In [21]:
points = 0

#this block is for a Male
if gender == "male":
    if age <= 34:
        points += -1
    if age >= 40 and age <= 44:
        points += 1
    if age >= 45 and age <= 49:
        points += 2
    if age >= 50 and age <= 54:
        points += 3
    if age >= 55 and age <= 59:
        points += 4
    if age >= 60 and age <= 64:
        points += 5
    if age >= 65 and age <= 69:
        points += 6
    if age >= 70:
        points += 7
    if LDL_val < 100:
        points += -3
    if LDL_val >= 160 and LDL_val <= 189:
        points += 1
    if LDL_val >= 190:
        points += 2
    if HDL_val < 35:
        points += 2
    if HDL_val >= 35 and LDL_val <= 44:
        points += 1
    if HDL_val >= 60:
        points += -1
    if sys <=129 and dia >= 85 and dia <= 89:
        points += 1
    if sys >= 130 and sys <=139 and dia < 90:
        points += 1
    if sys <= 139 and dia >= 90 and dia <= 99:
        points += 2
    if sys >= 140 and sys <=159 and dia < 100:
        points += 2
    if sys > 159 or dia > 99:
        points += 3
    if diabetes == True:
        points += 2
    if smoking == True:
        points += 2
        
    print("Male analysis complete")

Male analysis complete


In [22]:
# and this for a female
if gender == "female":
    if age <= 34:
        points += -9
    if age >= 35 and age <= 39:
        points += -4
    if age >= 45 and age <= 49:
        points += 3
    if age >= 50 and age <= 54:
        points += 6
    if age >= 55 and age <= 59:
        points += 7
    if age >= 60:
        points += 8
    if LDL_val < 100:
        points += -2
    if LDL_val >= 160:
        points += 2
    if HDL_val < 35:
        points += 5
    if HDL_val >= 35 and HDL_val <= 44:
        points += 2
    if HDL_val >= 45 and HDL_val <= 49:
        points += 1
    if HDL_val >= 60:
        points += -2
    if sys < 120 and dia < 80:
        points += -3
    if sys <=159 and dia >=90 and dia < 100:
        points += 2
    if sys > 159 or dia > 99:
        points += 3
    if diabetes == True:
        points += 4
    if smoking == True:
        points += 2
    print("Female analysis complete")

In [23]:
#this table converts from points to a percentage risk
male_results = {0:3,1:4,2:4,3:6,4:7,5:9,6:11,7:14,8:18,9:22,10:27,11:33,12:40,13:47}
female_results = {0:2,1:2,2:3,3:3,4:4,5:5,6:6,7:7,8:8,9:9,10:11,11:13,12:15,13:17,14:20,15:24,16:27}

if points >=14 and gender == "male":
    risk = ">= 56%"
elif points >=17 and gender == "female":
    risk = ">= 32%"
elif gender == "male":
    percent = male_results[points]
    risk = str(percent)+"%"
else:
    percent = female_results[points]
    risk = str(percent)+"%"
print("Total number of points: "+str(points))
print("Risk of Heart Disease: "+risk)

Total number of points: 11
Risk of Heart Disease: 33%


## Addendum: Getting data from a ValueSet

In [24]:
#Addendum: This is an an example of how you look up the values in a valueset, using the $expand operation
from fhirclient import client
import fhirclient.models.valueset as vs
# we specify the $expand operation. read_from() allows us to specify a relative path, so we can add the operation
# the operation is still against the vs model, as that is the resource type that is returned.
raceValueSet  = vs.ValueSet.read_from('ValueSet/omb-race-category/$expand',db.server)

#print out the expanded contents
print('All concepts in this ValueSet')
for concept in raceValueSet.expansion.contains:
    print(concept.display, '(', concept.code, concept.system,')')

print()
print("Only concepts with a 'w' in the display")    
raceValueSet  = vs.ValueSet.read_from('ValueSet/omb-race-category/$expand?filter=w',db.server)

#print out the expanded contents

for concept in raceValueSet.expansion.contains:
    print(concept.display, '(', concept.code, concept.system,')')


All concepts in this ValueSet
asked but unknown ( ASKU http://hl7.org/fhir/v3/NullFlavor )
unknown ( UNK http://hl7.org/fhir/v3/NullFlavor )

Only concepts with a 'w' in the display
asked but unknown ( ASKU http://hl7.org/fhir/v3/NullFlavor )
unknown ( UNK http://hl7.org/fhir/v3/NullFlavor )
